# Advanced optimizer tests

In [ ]:
// for local development
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt")' TextModels

In [ ]:
import Foundation
import TensorFlow
import TextModels
import x10_optimizers_optimizer
import PythonKit

In [ ]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

In [ ]:
let plt = Python.import("matplotlib.pyplot")

In [ ]:
var model = Dense<Float>(inputSize: 1000, outputSize: 100)

In [ ]:
let epochCount = 10
let stepsPerEpoch = 202 // function of training set size and batching configuration
let peakLearningRate: Float = 1e-3

In [ ]:
let beta1: Float = 0.9
let beta2: Float = 0.995
let useBiasCorrection = true

var optimizer = x10_optimizers_optimizer.GeneralOptimizer(
    for: model,
    TensorVisitorPlan(model.differentiableVectorView),
    defaultOptimizer: makeWeightDecayedAdam(
      learningRate: peakLearningRate,
      beta1: beta1,
      beta2: beta2
    )
)

var scheduledLearningRate = LinearlyDecayedParameter(
  baseParameter: LinearlyWarmedUpParameter(
      baseParameter: FixedParameter<Float>(peakLearningRate),
      warmUpStepCount: 20,
      warmUpOffset: 0),
  slope: -(peakLearningRate / Float(stepsPerEpoch * epochCount)),  // The LR decays linearly to zero.
  startStep: 10
)

In [ ]:
var steps = [Int]()
var lrs = [Float]()
var epochs = [Int]()

for epoch in 0..<epochCount {
    for epoch_step in 0..<stepsPerEpoch {
        // print("epoch: \(epoch), epoch_step: \(epoch_step)")
        let step = optimizer.step + 1 // for scheduled rates and bias correction, steps start at 1
        optimizer.learningRate = scheduledLearningRate(forStep: UInt64(step))
        if useBiasCorrection {
          let f_step = Float(step)
          optimizer.learningRate *= sqrtf(1 - powf(beta2, f_step)) / (1 - powf(beta1, f_step))
        }        
        steps.append(step)
        lrs.append(optimizer.learningRate)
        epochs.append(epoch)

        optimizer.step += 1
    }
}

In [ ]:
plt.figure(figsize: [15, 10])

plt.plot(steps, lrs)

plt.xlabel("Step")
plt.ylabel("Learning rate")
plt.title("ScheduledLearningRate")

plt.show()